# Общая настройка

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ./jars/postgresql-42.6.0.jar,./jars/clickhouse-jdbc-0.4.6.jar pyspark-shell'

In [2]:
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")

PG_NAME = os.getenv("POSTGRES_DB")
PG_PORT = os.getenv("POSTGRES_PORT")

CH_NAME = os.getenv("CLICKHOUSE_DB")
CH_PORT = os.getenv("CLICKHOUSE_PORT")

# Настройка подключения pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL") \
    .getOrCreate()

25/05/25 16:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
pg_jdbc_url = f"jdbc:postgresql://{HOST}:{PG_PORT}/{PG_NAME}"
pg_properties = {
    "user": USER,
    "password": PASSWORD,
    "driver": "org.postgresql.Driver"
}

In [5]:
ch_jdbc_url = f"jdbc:clickhouse://{HOST}:{CH_PORT}/{CH_NAME}"
ch_properties = {
    "user": USER,
    "password": PASSWORD,
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

# Витрины

In [6]:
from pyspark.sql.functions import *

In [7]:
def load_table(table_name):
    return spark.read.jdbc(url=pg_jdbc_url, table=table_name, properties=pg_properties)

In [8]:
d_customer = load_table("d_customer")
d_product = load_table("d_product")
d_seller = load_table("d_seller")
d_store = load_table("d_store")
d_sipplier = load_table("d_supplier")

f_sales = load_table("f_sales")

In [9]:
def save_report(df, name: str, order: str):
    df.write \
    .mode("overwrite") \
    .option("createTableOptions", f"""
        ENGINE = MergeTree()
        ORDER BY ({order})
    """) \
    .jdbc(url=ch_jdbc_url, table=name, properties=ch_properties)

In [39]:
def read_show(table_name:str):
    ch_df = spark.read.jdbc(url=ch_jdbc_url, table=table_name, properties=ch_properties)
    ch_df.show()

In [41]:
def save_show(df, name, order):
    save_report(df, name, order)
    read_show(name)

## 1. Витрина продаж по продуктам

### Топ-10 самых продаваемых продуктов

In [10]:
top_products = f_sales \
    .join(d_product, f_sales.sale_product_id == d_product.product_id) \
    .groupBy(
        d_product.product_id,
        d_product.product_name,
        d_product.product_category,
        d_product.pet_category,
        d_product.product_brand
    ) \
    .agg(
        sum("sale_quantity").alias("total_quantity_sold"),
        sum("sale_total_price").alias("total_revenue")
    ) \
    .orderBy(desc("total_revenue")) \
    .limit(10)

In [42]:
save_show(top_products, "1_top10products", "product_id")

+----------+------------+----------------+------------+-------------+-------------------+--------------------+
|product_id|product_name|product_category|pet_category|product_brand|total_quantity_sold|       total_revenue|
+----------+------------+----------------+------------+-------------+-------------------+--------------------+
|       260|   Bird Cage|             Toy|        Cats|       Voonix|                  4|499.6900000000000...|
|       262|   Bird Cage|             Toy|        Fish|      Tagopia|                  9|499.6200000000000...|
|       419|     Cat Toy|             Toy|        Dogs|        Lazzy|                 10|499.7300000000000...|
|      2595|    Dog Food|             Toy|        Dogs|       Divavu|                  8|499.7600000000000...|
|      2955|    Dog Food|            Food|        Dogs|         Mydo|                  2|499.7600000000000...|
|      3792|     Cat Toy|             Toy|        Dogs|        Vidoo|                  7|499.8500000000000...|
|

25/05/25 16:35:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:35:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:35:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [09d43860-e52d-4cbf-8963-dbec8626452b] (2 queries & 0 savepoints) is committed.
25/05/25 16:35:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [455e6451-7b2b-43a4-a276-a96d02db0687] (0 queries & 0 savepoints) is committed.


### Общая выручка по категориям продуктов

In [ ]:
revenue_by_category = f_sales \
    .join(d_product, f_sales.sale_product_id == d_product.product_id) \
    .groupBy(
        d_product.product_category,
        d_product.pet_category
    ) \
    .agg(
        sum("sale_total_price").alias("total_revenue"),
        count("sale_id").alias("total_transactions"),
        sum("sale_quantity").alias("total_units_sold")
    ) \
    .orderBy(desc("total_revenue"))

In [43]:
save_show(revenue_by_category, "1_bycategory", "product_category")

+----------------+------------+--------------------+------------------+----------------+
|product_category|pet_category|       total_revenue|total_transactions|total_units_sold|
+----------------+------------+--------------------+------------------+----------------+
|            Cage|       Birds|181830.9300000000...|               693|            3740|
|            Cage|    Reptiles|172473.9200000000...|               670|            3672|
|            Cage|        Dogs|166057.7100000000...|               678|            3736|
|            Cage|        Fish|158665.9600000000...|               657|            3491|
|            Cage|        Cats|152089.4200000000...|               629|            3418|
|            Food|       Birds|177085.2300000000...|               675|            3708|
|            Food|        Fish|170664.5800000000...|               676|            3741|
|            Food|        Cats|165220.8100000000...|               637|            3593|
|            Food|   

25/05/25 16:36:19 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:36:19 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:36:19 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [e4f47d1e-ff0c-46d9-b823-3265f104dfb8] (2 queries & 0 savepoints) is committed.
25/05/25 16:36:19 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [799db1d7-54e4-440d-b852-8cbb0fc2fe24] (0 queries & 0 savepoints) is committed.


### Средний рейтинг и количество отзывов для каждого продукта

In [45]:
product_ratings = d_product.select(
    "product_id",
    "product_rating",
    "product_reviews"
)

In [46]:
save_show(product_ratings, "1_ratings", "product_id")

25/05/25 16:40:52 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:40:52 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:40:52 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [8edb19f2-be76-4472-bf6e-1bbbce5d5685] (11 queries & 0 savepoints) is committed.
25/05/25 16:40:52 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [1b47e9bf-baa8-442b-a200-36b1274d9554] (0 queries & 0 savepoints) is committed.


+----------+--------------------+---------------+
|product_id|      product_rating|product_reviews|
+----------+--------------------+---------------+
|      6620|1.900000000000000000|            874|
|      6622|1.800000000000000000|            439|
|      6624|4.400000000000000000|            549|
|      6625|2.800000000000000000|            696|
|      6627|3.500000000000000000|            547|
|      6629|4.300000000000000000|            631|
|      6630|1.200000000000000000|            229|
|      6632|1.300000000000000000|            942|
|      6635|2.800000000000000000|            535|
|      6637|3.300000000000000000|            552|
|      6639|1.300000000000000000|            183|
|      6641|2.200000000000000000|            620|
|      6642|1.800000000000000000|              8|
|      6644|4.200000000000000000|             34|
|      6645|3.900000000000000000|            188|
|      6647|2.700000000000000000|            320|
|      6649|5.000000000000000000|            718|


## 2. Витрина продаж по клиентам

### Топ-10 клиентов с наибольшей общей суммой покупок

In [ ]:
top_customers = f_sales \
    .join(d_customer, f_sales.sale_customer_id == d_customer.customer_id) \
    .groupBy(
        d_customer.customer_id,
        d_customer.customer_first_name,
        d_customer.customer_last_name,
        d_customer.customer_email,
        d_customer.customer_country
    ) \
    .agg(
        sum("sale_total_price").alias("total_spent"),
        count("sale_id").alias("purchase_count"),
        sum("sale_quantity").alias("total_items_purchased")
    ) \
    .orderBy(desc("total_spent")) \
    .limit(10)

In [48]:
save_show(top_customers, "2_top10customers", "customer_id")

25/05/25 16:42:38 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:42:39 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:42:39 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [c4909de0-79dc-40f9-a791-384e075f9bbe] (2 queries & 0 savepoints) is committed.
25/05/25 16:42:39 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [0bab8df1-687a-4157-92a0-a0b6037e8af5] (0 queries & 0 savepoints) is committed.


+-----------+-------------------+------------------+--------------------+----------------+--------------------+--------------+---------------------+
|customer_id|customer_first_name|customer_last_name|      customer_email|customer_country|         total_spent|purchase_count|total_items_purchased|
+-----------+-------------------+------------------+--------------------+----------------+--------------------+--------------+---------------------+
|       1494|              Hayes|            McKain|sstappardbp@busin...|        Portugal|499.8000000000000...|             1|                    9|
|       1527|           Isahella|            Colley|bselewayi0@chron.com|          Russia|499.6900000000000...|             1|                    4|
|       1795|            Lavinia|         Horsburgh|previllh3@tinyurl...|          Poland|499.7300000000000...|             1|                   10|
|       4466|                Ava|             Lomas|dsorea0@geocities...|           China|499.760000000000

### Распределение клиентов по странам

In [ ]:
customers_by_country = f_sales \
    .join(d_customer, f_sales.sale_customer_id == d_customer.customer_id) \
    .groupBy(d_customer.customer_country) \
    .agg(
        count("customer_id").alias("customer_count"),
        sum("sale_total_price").alias("total_revenue"),
        avg("sale_total_price").alias("avg_revenue_per_customer"),
        sum("sale_quantity").alias("total_items_sold")
    ) \
    .orderBy(desc("total_revenue"))

In [50]:
save_show(customers_by_country, "2_contryspread", "customer_country")

+-------------------+--------------+--------------------+------------------------+----------------+
|   customer_country|customer_count|       total_revenue|avg_revenue_per_customer|total_items_sold|
+-------------------+--------------+--------------------+------------------------+----------------+
|        Afghanistan|            31|8532.720000000000...|    275.2490322580645...|             171|
|      Aland Islands|             2|571.4600000000000...|    285.7300000000000...|               7|
|            Albania|            46|11821.24000000000...|    256.9834782608695...|             227|
|     American Samoa|             3|385.7500000000000...|    128.5833333333333...|              22|
|            Andorra|             2|579.8500000000000...|    289.9250000000000...|              15|
|             Angola|             6|1914.940000000000...|    319.1566666666666...|              30|
|Antigua and Barbuda|             3|775.3700000000000...|    258.4566666666666...|              19|


25/05/25 16:43:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:43:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:43:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [953b97c6-28ba-414d-beca-0ac8cb4931cc] (2 queries & 0 savepoints) is committed.
25/05/25 16:43:51 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [20932a02-ffed-4b31-a6bf-d48dc6ded59b] (0 queries & 0 savepoints) is committed.


### Средний чек для каждого клиента

In [51]:
customer_avg_check = f_sales \
    .join(d_customer, f_sales.sale_customer_id == d_customer.customer_id) \
    .groupBy(
        d_customer.customer_id,
    ) \
    .agg(
        (sum("sale_total_price") / count("sale_id")).alias("avg_check"),
    )

In [52]:
save_show(customer_avg_check, "2_customeravgcheck", "customer_id")

25/05/25 16:46:12 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:46:12 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 16:46:12 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [42397381-7db2-4e51-af63-85e8485ad8e2] (11 queries & 0 savepoints) is committed.
25/05/25 16:46:12 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [aa269b74-2414-428c-a1b0-ac4325f8c99a] (0 queries & 0 savepoints) is committed.


+-----------+--------------------+
|customer_id|           avg_check|
+-----------+--------------------+
|         10|367.7300000000000...|
|         24|335.3000000000000...|
|         25|272.8300000000000...|
|         38|270.4100000000000...|
|         45|205.5400000000000...|
|         50|179.0700000000000...|
|         62|14.26000000000000...|
|         63|309.2900000000000...|
|         70|46.43000000000000...|
|         73|361.7200000000000...|
|         77|131.5800000000000...|
|         80|425.8700000000000...|
|         82|416.7800000000000...|
|        102|376.8900000000000...|
|        113|353.5900000000000...|
|        121|103.6800000000000...|
|        125|182.8500000000000...|
|        156|80.26000000000000...|
|        160|167.1200000000000...|
|        167|306.9400000000000...|
+-----------+--------------------+
only showing top 20 rows


## 3. Витрина продаж по времени

### Месячные и годовые тренды продаж

In [53]:
sales_trends = f_sales \
    .withColumn("year", year("sale_date")) \
    .withColumn("month", month("sale_date")) \
    .groupBy("year", "month") \
    .agg(
        sum("sale_total_price").alias("monthly_revenue"),
        count("sale_id").alias("order_count"),
        sum("sale_quantity").alias("total_items_sold"),
        (sum("sale_total_price") / count("sale_id")).alias("avg_order_value"),
        approx_count_distinct("sale_customer_id").alias("unique_customers")
    ) \
    .orderBy("year", "month")

In [54]:
save_show(sales_trends, "3_salestrends", "year, month")

25/05/25 17:01:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+--------------------+-----------+----------------+--------------------+----------------+
|year|month|     monthly_revenue|order_count|total_items_sold|     avg_order_value|unique_customers|
+----+-----+--------------------+-----------+----------------+--------------------+----------------+
|2010|    1|203842.2000000000...|        807|            4478|252.5925650557620...|             802|
|2011|    1|197096.2200000000...|        775|            4171|254.3177032258064...|             811|
|2012|    1|202942.4800000000...|        800|            4534|253.6781000000000...|             821|
|2013|    1|199344.9800000000...|        788|            4257|252.9758629441624...|             765|
|2014|    1|191980.7200000000...|        773|            4277|248.3579818887451...|             765|
|2015|    1|197908.7900000000...|        760|            4043|260.4063026315789...|             734|
|2016|    1|192888.2600000000...|        766|            4213|251.8123498694516...|        

25/05/25 17:01:38 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 17:01:38 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/25 17:01:38 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [83f8236c-43c6-45af-9012-dce5969e43ec] (2 queries & 0 savepoints) is committed.
25/05/25 17:01:38 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [5c311a73-bb67-4479-a63d-73aaf0471572] (0 queries & 0 savepoints) is committed.


### Сравнение выручки за разные периоды

### Средний размер заказа по месяцам

## 4. Витрина продаж по магазинам

### Топ-5 магазинов с наибольшей выручкой

### Распределение продаж по городам и странам

### Средний чек для каждого магазина

## 5. Витрина продаж по поставщикам

### Топ-5 поставщиков с наибольшей выручкой

### Средняя цена товаров от каждого поставщика

### Распределение продаж по странам поставщиков

## 6. Качество продукции

### Продукты с наивысшим и наименьшим рейтингом

### Корреляция между рейтингом и объемом продаж

### Продукты с наибольшим количеством отзывов

# Пример кода

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL and ClickHouse") \
    .getOrCreate()

print(spark.sparkContext._jsc.sc().listJars())

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
pg_jdbc_url = "jdbc:postgresql://localhost:5432/db"
properties = {
    "user": "Maxim",
    "password": "12345",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=pg_jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=pg_jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# Записываем данные в ClickHouse через JDBC
ch_jdbc_url = "jdbc:clickhouse://localhost:8123/default"
properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "Maxim",
    "password": "12345"
}

# Записываем данные в ClickHouse
aggregated_df.write.jdbc(url=ch_jdbc_url, table="default.report_ch", mode="append", properties=properties)

# Читаем данные из ClickHouse
ch_df = spark.read.jdbc(url=ch_jdbc_url, table="default.report_ch", properties=properties)

# Создаем временное view
ch_df.createOrReplaceTempView("report_ch_view")

ch_pandas_df = spark.sql("select Total_Salary from report_ch_view where Department = 'HR' limit 1").toPandas()

print("Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:")
display(ch_pandas_df)

# 6. Останавливаем SparkSession
spark.stop()